# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD3100 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap,ee
import matplotlib.colors as colors
from shapely.geometry import shape, box
from matplotlib import colors
import matplotlib.patches as mpatches
import contextily as ctx

## 1. Primeros pasos en Google Earth Engine.

Para utilizar la plataforma GEE, primero es necesario crear una cuenta de usuario:

https://code.earthengine.google.com/register

### 1.1 Instalación de librerías.

Una vez registrado, puede utilizar el Code Editor, una interfaz web (IDE) de programación en Java: https://earthengine.google.com/platform/.

O también, puede utilizarse la API de Python para GEE, mediante librerías de cliente:`ee`y `geemap`.

#### `ee`: The Python API package is called ee.

https://github.com/google/earthengine-api

Para instalar: `!pip install earthengine-api`

Tutoriales y ejemplos: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

#### *`geemap`: A Python package for interactive geospatial analysis and visualization with Google Earth Engine

https://geemap.org/

Para instalar: `pip install geemap`


### 1.2 Autentificación e inicialización.


Una vez instalada la librería `ee`, sólo se requiere importarla y autentificarse en la plataforma (con su cuenta de usuario previamente creada), de manera de generar un token o código de acceso.

Estos pasos de autentificación deben completarse cada vez que se inicia una sesión de trabajo, si se reinicia el Kernel del Notebook, o si el Kernel se cae o desconecta por inactividad.

Para mayor información, ver:

https://developers.google.com/earth-engine/guides/auth

In [ ]:
#ee.Authenticate()

In [ ]:
ee.Initialize(project='ee-aguirrepaula')  #cambiar por su nombre de proyecto

## 1.3 Funciones de interés.

En este tutorial, se aplicarán las siguiente funciones de Earth Engine (`ee`) y `geemap`:

- imageCollection: https://developers.google.com/earth-engine/apidocs/ee-imagecollection?hl=es-419
- image: https://developers.google.com/earth-engine/apidocs/ee-image?hl=es-419
- normalizedDifference: https://developers.google.com/earth-engine/apidocs/ee-image-normalizeddifference?hl=es-419
- Mapa interactivo con geemap: https://geemap.org/notebooks/00_geemap_key_features/
- splitMap: https://geemap.org/notebooks/04_split_panel_map/

## 2. Zona de estudio.

Para este ejercicio, estudiaremos la zona de Viña del Mar, en la quinta región de Chile. El objetivo final, es analizar el área quemada en el incendio ocurrido el 24 de diciembre de 2022.

## 3. Selección de imágenes Sentinel 2 desde GEE.

El catálogos de GEE contiene numerosos catálogoso de datos y productos de distintas misiones de observación terrestre. En primer lugar, utilizaremos datos multiespectrales de Sentinel 2:

https://developers.google.com/earth-engine/datasets/catalog/sentinel-2


Para seleccionar las imágenes a utilizar en un determinado análisis, podemos filtar este catálogo por:


- Área geográfica
- Fecha
- Cobertura de nubes de la imagen
- Otras especificaciones técnicas (Ej. órbita, valores, nombre de imagen, etc.)

https://developers.google.com/earth-engine/apidocs/ee-filter



### 3.1 Filtrado por área geográfica.

Para filtrar geográficamente el catálogo, es necesario definir una AOI, o "area of interest".

In [ ]:
minx,miny,maxx,maxy=-71.6,-33.1,-71.5,-33,
xy=[[minx,miny],[maxx,miny],[maxx,maxy],[minx,maxy]]
print(xy)
aoi = ee.Geometry.Polygon(xy)
aoi

In [ ]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi)

### 3.2 Filtrado por fechas.

In [ ]:
dates=['2022-12-15','2023-01-15']

collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi).filterDate(dates[0],dates[1])
images=collection.aggregate_array('system:id').getInfo()
images #lista de imagenes seleccionadas

#### Las operaciones anteriores pueden resumirse en las siguientes funciones:

In [ ]:
def getAOI_latlon(dat,crs=4326):
    minx,miny,maxx,maxy=dat.to_crs(crs).geometry.total_bounds
    xy=[[minx,miny],[maxx,miny],[maxx,maxy],[minx,maxy]]
    aoi = ee.Geometry.Polygon(xy)
    return(aoi)

def getImage(aoi,dates,clouds=20,mosaic=False):
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi).filterDate(dates[0],dates[1])
    collection = collection.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',clouds));
    images=collection.aggregate_array('system:id').getInfo()

    return(images)


In [ ]:
images=getImage(aoi,dates,clouds=20,mosaic=True) 
len(images)

In [ ]:
im=ee.Image(images[-3])
im


In [ ]:
Map = geemap.Map(center=[-33.05,-71.5], zoom=12)
vis_params = {"min": 0, "max": 3000, "bands": ["B4", "B3", "B2"]}

Map.addLayer(im.clip(aoi), vis_params,name='Image')
Map.addLayer(aoi,name='aoi',shown=False)
Map

In [ ]:
## Corrección por nubes y escalamiento

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [ ]:
im_clean= mask_s2_clouds(im)
im_clean

In [ ]:
Map = geemap.Map(center=[-33.05,-71.5], zoom=12)
vis_params = {"min": 0, "max": 0.4, "bands": ["B4", "B3", "B2"]}

Map.addLayer(im_clean.clip(aoi), vis_params,name='Image')
Map

## 4. Cálculo de índices espectrales

### 4.1 Calcular y visualizar el NDVI (para una imagen, y para la mediana mensual de un mes a su elección ).

In [ ]:
image=ee.Image(images[0])
image=mask_s2_clouds(image)
ndvi= image.normalizedDifference(['B8','B4']);
clipped_ndvi=ndvi.clip(aoi)


In [ ]:
Map = geemap.Map(center=[-33.05,-71.5], zoom=12)
ndviParams = {"min": -1, "max": 1, "palette": ['blue', 'white', 'green']};
Map.addLayer(clipped_ndvi, ndviParams,name='NDVI')
Map.addLayer(aoi,name='aoi',shown=False)
Map

### 4.4 Genere los raster de NBR y NDVI para cada fecha, y dNBR.


In [ ]:
im_pre=mask_s2_clouds(ee.Image(images[0]))
im_post=mask_s2_clouds(ee.Image(images[-1]))

ndvi_pre= im_pre.normalizedDifference(['B8','B4']).clip(aoi)
ndvi_post= im_post.normalizedDifference(['B8','B4']).clip(aoi)


In [ ]:
nbr_pre= im_pre.normalizedDifference(['B8','B12']).clip(aoi)
nbr_post= im_post.normalizedDifference(['B8','B12']).clip(aoi)
dnbr=nbr_pre.subtract(nbr_post)
geemap.ee_export_image(dnbr, filename='dnbr.tif',crs='EPSG:32719', scale=10,region=aoi)

### 4.5 Genere un mapa con las siguientes capas: imagen RGB, NDVI pre y post incendio, y dNBR. Utilice paletas de colores coherentes con la naturaleza de sus datos (Ej: vegetación en gama de verdes). ¿Puede identificar las zonas quemadas y cambios en el estado de la vegetación?. *Guarde su mapa en formato html*.


In [ ]:
Map = geemap.Map(center=[-33.05,-71.5], zoom=12)
ndviParams = {"min": -1, "max": 1, "palette": ['blue', 'white', 'green']};
Map.addLayer(ndvi_pre, ndviParams,name='NDVI Pre')
Map.addLayer(ndvi_post, ndviParams,name='NDVI Post')

nbrParams = {"min": 0, "max": 1, "palette": ['white', 'orange', 'brown']};

Map.addLayer(nbr_pre, nbrParams,name='NBR Pre')
Map.addLayer(nbr_post, nbrParams,name='NBR Post')
Map.addLayer(dnbr, nbrParams,name='dNBR')
Map.to_html(filename='map.html', title="Mapa Incendios", width="100%", height="880px")

Map

### 4.6 Genere un *split map* con las capas de NDVI o NBR pre- y post-incendio.


In [ ]:
Map = geemap.Map(center=[-33.05,-71.5], zoom=13)
vis_params = {"min": 0, "max": 0.3, "bands": ["B4", "B3", "B2"]}


left_layer = geemap.ee_tile_layer(im_pre, vis_params, "NDVI pre")
right_layer = geemap.ee_tile_layer(im_post, vis_params, "NDVI post")

Map.split_map(left_layer, right_layer)

Map